In [1]:
# Imports
import os
import time
import glob
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imutils.video import VideoStream
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import Bunch
import pandas as pd
from functions import getLargestContour, getContourFeatures, unpackAdvancedFeatures, displayImages


In [2]:
# Constants
data_path = 'images'
mappings = {"r": "r", "p": "p", "s": "s"}
frame_size = (612, 816)
feature_names = [
    'area',
    'perimeter',
    'aspect_ratio',
    'extent',
    'convex_hull_area',           # Area of the convex hull
    'hull_perimeter',              # Perimeter of the convex hull
    'solidity',                    # Ratio of area to convex hull area
    'circularity',                 # Measure of how circular the shape is
    'eccentricity',                # Measure of the elongation of the shape
    'num_convexity_defects',       # Number of convexity defects
    'max_defect_depth',            # Maximum depth of convexity defects
    'major_axis_length',           # Length of the major axis of the fitted ellipse
    'minor_axis_length',           # Length of the minor axis of the fitted ellipse
    'orientation',                  # Orientation angle of the fitted ellipse
    'hu_moment',
    'centroid_x',
    'centroid_y'
]


categories = ['Rock', 'Paper', 'Scissor']
size = (612, 816)

lower_skin = np.array([0, 20, 70], dtype=np.uint8)  # Lower boundary of skin tone in HSV
upper_skin = np.array([20, 255, 255], dtype=np.uint8)  # Upper boundary of skin tone in HSV




In [3]:
# Initialize lists
data = np.empty((0, len(feature_names)), float)  # To store feature vectors
target = []  # To store labels
images = []  # List to store images
labels = []  # List to store corresponding labels

In [4]:
# Key mappings
kMappings = {ord(key): value for key, value in mappings.items()}

In [5]:
for category in categories:
    # Get the corresponding label from the mappings dictionary
    label = mappings[category[0].lower()]  # 'r' -> 0, 'p' -> 1, 's' -> 2
    
    # Form the path to the current category's images
    folder_path = os.path.join(data_path, category)
    
    # Get all image file paths in the current folder
    image_paths = glob.glob(os.path.join(folder_path, '*.jpg'))
    
    print(f"[INFO] Processing {category} images, found: {len(image_paths)}")
    
    # Loop over each image path
    for image_path in image_paths:
        # Load the image using OpenCV
        img = cv.imread(image_path)
        
        if img is None:
            print(f"[WARNING] Unable to load image {image_path}. Skipping.")
            continue
        
        # Resize the image to a standard frame size
        img = cv.resize(img, frame_size)
        
        # Append the image and its label
        images.append(img)  # Store the image
        labels.append(label)  # Store the corresponding label

# Convert images and labels to NumPy arrays for consistency
images = np.array(images)
labels = np.array(labels)

# Display some information about the dataset
print(f"[INFO] Number of images loaded: {len(images)}")
print(f"[INFO] Shape of images array: {images.shape}")
print(f"[INFO] Number of labels: {len(labels)}")

[INFO] Processing Rock images, found: 54
[INFO] Processing Paper images, found: 40
[INFO] Processing Scissor images, found: 39
[INFO] Number of images loaded: 133
[INFO] Shape of images array: (133, 816, 612, 3)
[INFO] Number of labels: 133


In [6]:
# Loop over each image and process for background removal
for i, img in enumerate(images):
    # Convert the image to the HSV color space
    hsv_img = cv.cvtColor(img, cv.COLOR_BGR2HSV)

    # Create a binary mask where skin color is within the specified range
    masked_img = cv.inRange(hsv_img, lower_skin, upper_skin)

    # Apply morphological operations to clean up the mask (remove small noises)
    kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (5,5))
    masked_img = cv.morphologyEx(masked_img, cv.MORPH_CLOSE, kernel)
    masked_img = cv.erode(masked_img, None, iterations=2)
    masked_img = cv.dilate(masked_img, None, iterations=2)

    # Create the result image by applying the mask to the original image
    masked_color_img = cv.bitwise_and(img, img, mask=masked_img)

    # Convert the result to grayscale to proceed with feature extraction
    gray_img = cv.cvtColor(masked_color_img, cv.COLOR_BGR2GRAY)

    # Apply Gaussian blur to the grayscale image to reduce noise
    blurred_img = cv.GaussianBlur(gray_img, (5,5), 0)

    # Apply binary thresholding to convert the image to binary (black & white)
    _, bw_img = cv.threshold(blurred_img, 10, 255, cv.THRESH_BINARY_INV)

    # Apply morphological operations to remove small noise and close gaps
    bw_img = cv.morphologyEx(bw_img, cv.MORPH_CLOSE, kernel)

    # Apply high-pass filter to remove low-frequency background
    bw_img = cv.morphologyEx(bw_img, cv.MORPH_GRADIENT, kernel)

    # Check if any foreground remains after thresholding
    if cv.countNonZero(bw_img) == 0:
        print(f"[WARNING] No foreground detected in image {i}. Skipping.")
        continue

    # Find the largest contour from the binary image
    try:
        contour = getLargestContour(bw_img)
    except ValueError:
        print(f"[WARNING] Could not find contours in image {i}. Skipping.")
        continue

    # Extract features from the largest contour
    features, defects, feature_vector = getContourFeatures(contour)
    
    # Append the features to the data matrix
    data = np.append(data, np.array([features]), axis=0)
    
    # Append the corresponding label from the 'labels' array
    target.append(labels[i])

    displayImages([img, hsv_img, masked_img, masked_color_img, gray_img, blurred_img, bw_img], 
                  [img],
                  contour, defects, frame_size)

    print("[INFO] contour features: {}".format(feature_vector))

    # Wait for a key press and quit if 'q' is pressed
    k = cv.waitKey(1) & 0xFF
    if k == ord("q"):
        break

# Convert the target list to a NumPy array
target = np.array(target)

# Display the shape of the resulting feature dataset
print(f"[INFO] Feature extraction complete.")
print(f"[INFO] Shape of data matrix: {data.shape}")
print(f"[INFO] Number of target labels: {target.shape[0]}")

# Create a Bunch object containing the dataset
dataset = Bunch(data=data,
                target=target,
                feature_names=feature_names,
                unique_targets=np.unique(target))

print(f"[INFO] Dataset created with shape: {dataset.data.shape}")

[INFO] contour features: [ 4.50000000e+02  3.54804688e+01  2.51445312e+01  1.44179688e+01
  1.15078125e+01  8.35937500e+00  3.57750000e+03  2.44826527e+03
  1.20744681e+00  2.09573296e-02  1.15917500e+05  1.33365580e+03
  3.08624668e-02  7.50019511e-03  7.93075841e-01  2.90000000e+01
  4.50000000e+02  4.82628448e+02  2.93980042e+02  1.13097420e+02
 -9.70789860e-01  3.61000000e+02  5.18000000e+02]
[INFO] contour features: [ 4.60000000e+02  3.55585938e+01  2.07226562e+01  1.38671875e+01
  1.36015625e+01  9.62109375e+00  3.47700000e+03  2.44102769e+03
  1.46835443e+00  2.37137167e-02  1.17373500e+05  1.31932535e+03
  2.96233818e-02  7.33278825e-03  7.79458648e-01  2.60000000e+01
  4.60000000e+02  4.75232941e+02  2.97711426e+02  9.59212952e+01
 -9.81204019e-01  3.46000000e+02  4.68000000e+02]
[INFO] contour features: [ 4.65000000e+02  3.57500000e+01  2.16210938e+01  1.48632812e+01
  1.35781250e+01  9.90625000e+00  3.62950000e+03  2.50344695e+03
  1.41265060e+00  2.33096565e-02  1.27532500e

In [7]:
unpackAdvancedFeatures(dataset.data, dataset.target)

[INFO] Number of feature vectors: 133
[INFO] Number of labels: 133
Image #   Area           Perimeter      Aspect Ratio   Extent         Convex Hull Area    Hull Perimeter      Solidity       Circularity    Eccentricity   Convexity Defects   Max Defect Depth    Major Axis Length   Minor Axis Length   Orientation    Hu moments     Centroid_x     Centroid_y     Label
0         3577.50        2448.27        1.21           0.02           115917.50           1333.66             0.03           0.01           0.79           29.0                450.00              482.63              293.98              113.10         -0.97          361.00         518.00         r
1         3477.00        2441.03        1.47           0.02           117373.50           1319.33             0.03           0.01           0.78           26.0                460.00              475.23              297.71              95.92          -0.98          346.00         468.00         r
2         3629.50        2503.45      

In [8]:
# Export to csv
"""Export the feature data and target labels to a CSV file."""
# Create a DataFrame from the feature data
f = pd.DataFrame(data, columns=feature_names)  # Use the feature_names list
# Ad the target labels as a new column
f['Label'] = target
# Expot to CSV
filename = 'data/exported_features.csv'
f.to_csv(filename, index=False)
print(f"[INFO] Data exported to {filename}")

[INFO] Data exported to data/exported_features.csv
